![](lcc.jpg)

# Preprecesamiento de datos con `scikit-learn`

## Reconocimiento de Patrones, 2024-2

## Licenciatura en Ciencia de la Computación

**Julio Waissman**

[Abrir en google Colab](https://colab.research.google.com/github/ml-unison/ml-unison.github.io/blob/main/ejemplos/preprocesamiento.ipynb)

Este ejemplo ilustra cómo aplicar diferentes procesos de preprocesamiento y extracción de características a diferentes subconjuntos de características, utilizando
`ColumnTransformer`. Esto es particularmente útil para el
caso de conjuntos de datos que contienen tipos de datos heterogéneos, ya que es posible que queramos
escalar las características numéricas y codificar con `one-hot` las categóricas.

En este ejemplo, los datos numéricos se escalan de manera estándar, después imputar los valores faltantes usando la media. Los datos categóricos se codifican a través de `OneHotEncoder`, que
crea una nueva categoría para los valores faltantes.

Además, mostramos dos formas diferentes de enviar las columnas al preprocesador en particular: por nombres de columna y por tipos de datos de columna.

Por último, el proceso de preprocesamiento se integra en un proceso de predicción completo
utilizando `pipeline.Pipeline`, junto con un modelo de clasificación simple, el cual para este caso no es importante.

Una adaptación de [esta libreta](https://scikit-learn.org/stable/_downloads/26f110ad6cff1a8a7c58b1a00d8b8b5a/plot_column_transformer_mixed_types.ipynb) de Pedro Morales (<part.morales@gmail.com>).

In [1]:
import numpy as np

from sklearn.datasets import fetch_openml # Bajar los datos
from sklearn.preprocessing import OneHotEncoder # Codificar variables categóricas 
from sklearn.preprocessing import StandardScaler # Estandarizar los datos numéricos
from sklearn.impute import SimpleImputer # Imputar valores faltantes
from sklearn.feature_selection import SelectPercentile, chi2 # Seleccionar variables

from sklearn.compose import ColumnTransformer # Unir transformaciones
from sklearn.pipeline import Pipeline # Crear un pipeline `para que sea un solo modelo

from sklearn.linear_model import LogisticRegression # Modelo de regresión logística
from sklearn.model_selection import train_test_split # Dividir los datos en entrenamiento y prueba

np.random.seed(0) # Semilla para reproducibilidad

### Cargando los datos

Cargar datos del Titanic desde https://www.openml.org/d/40945

In [2]:
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

In [3]:
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [4]:
y

0       1
1       1
2       0
3       0
4       0
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: survived, Length: 1309, dtype: category
Categories (2, object): ['0', '1']

Para entrenar nuestro clasificador necesitamos procesar los datos en función del origen de sus características:

1. Características numéricas:
    - age: float;
    - `fare`: float.

2. Características categóricas:
    - `embarked`: categoría codificadas como cadenas {'C', 'S', 'Q'};
    - `sex`: categorías codificadas como cadenas {'female', 'male'};
    - `pclass`: números enteros ordinales {1, 2, 3}.
    - 
Creamos los canales de preprocesamiento para datos numéricos y categóricos. Ten en cuenta que `pclass` puede tratarse como una característica categórica o numérica.

Cada serie de pasos bien establecidos, vamos a definirlos con `Pipeline`. Empecemos con `SimpleImputer` seguido de un `StandarScaler`:

In [5]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ("scaler", StandardScaler())
    ]
)

Y ahora un `Pipeline`con un `OneHotEncoding` y un `Selector`:

In [6]:
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)

Y creamos un producto por columnas, empaquetado, especificando a cuales entradas les aplicamos que proceso. 

In [7]:
numeric_features = ["age", "fare"]
categorical_features = ["embarked", "sex", "pclass"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Y ahora vamos a crear un modelo compuesto de su parte de preprocesamiento, seguido de un modelo de aprendizaje en caja negra.

In [12]:
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", LogisticRegression())
    ]
)

Se separan datos de entrenamiento de los de prueba:

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=0
)

Y vemos el flujo de datos,

In [16]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x13e364400>))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

Y por último se vuelve a entrenar el modelo con todos los datos, y se prueba con datos nunca usados en el entrenamiento:

In [15]:
clf.fit(X_train, y_train)
print(f"model score: {clf.score(X_test, y_test)}")

model score: 0.7977099236641222


Podemos ver directamente como se encuentran los bloques relacionados entre sí visualizando `clf`. 

In [18]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x13e364400>))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])